In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from blazeface.dataset import input_dataset

# Load data

In [ ]:
# take small subset of the training set to analyze
data_train, info = input_dataset.load_the300w_lp(split="train[:10%]")

In [ ]:
for x in data_train.take(1):
    break

print(x.keys())

# Visualize raw labels

In [ ]:
def visualize_landmarks(sample, ax, landmarks=None):
    """
    Args:
        sample (dict): must contain keys 'image' and 'landmarks_2d'.
        ax (AxesSubplot):
        landmarks (np.ndarray): if given, it will override 'landmarks_2d' in sample.
    """
    img = sample['image']
    if landmarks is None:
        landmarks = sample['landmarks_2d'].numpy()
    shape = tf.shape(img).numpy()
    ax.scatter(landmarks[:,0] * shape[0], landmarks[:,1] * shape[1], alpha=0.6, s=2, c='red');

In [ ]:
n_rows = 5
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axes = np.ravel(axes)

for i, x in enumerate(data_train.take(n_rows * n_cols)):
    ax = axes[i]
    ax.imshow(x['image'])
    visualize_landmarks(x, ax)

# Visualize preprocessed data (inputs to BlazeFace)

In [ ]:
def visualize_bbox(sample, ax):
    img = sample['image']
    shape = tf.shape(img).numpy()
    x1, y1, x2, y2 = input_dataset.landmarks_to_bboxes(x['landmarks_2d']).numpy()
    x1 *= shape[1]
    y1 *= shape[0]
    x2 *= shape[1]
    y2 *= shape[0]
    rect = Rectangle((x1, y1), x2 - x1, y2 - y1, fc="None", ec='green')
    ax.add_patch(rect)

In [ ]:
n_rows = 5
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axes = np.ravel(axes)

for i, x in enumerate(data_train.take(n_rows * n_cols)):
    ax = axes[i]
    ax.imshow(x['image'])
    # visualize bbox
    visualize_bbox(x, ax)
    # visualize landmarks
    landmarks_2d = input_dataset.reduce_landmarks(x['landmarks_2d']).numpy()
    visualize_landmarks(x, ax, landmarks=landmarks_2d)

# Create input dataset

In [ ]:
    img = image_data["image"]
    img = tf.image.convert_image_dtype(img, tf.float32)
    gt_landmarks = tf.expand_dims(image_data["landmarks_2d"], 0)
    gt_boxes = generate_bboxes_from_landmarks(gt_landmarks)
    gt_landmarks = filter_landmarks(gt_landmarks)
    img = tf.image.resize(img, (final_height, final_width))
    if augmentation_fn:
        img, gt_boxes, gt_landmarks = augmentation_fn(img, gt_boxes, gt_landmarks)
    img = (img - 0.5) / 0.5
    return img, gt_boxes, gt_landmarks

In [ ]:
def unpack_dct(dct):
    return dct['image'], dct['landmarks_2d']


def preprocess_image(img, landmarks):
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, (128, 128), preserve_aspect_ratio=True)
    return img, landmarks

In [ ]:
ds = data_train
ds = ds.map(unpack_dct)
ds = ds.map(preprocess_image)
ds = ds.map(lambda img, lmarks: (img, input_dataset.landmarks_to_bboxes(lmarks), input_dataset.reduce_landmarks(lmarks)))

ds = ds.batch(8)

In [ ]:
for temp in ds.take(1):
    break

[element.shape for element in temp]